In [1]:
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from scikitplot import plotters as skplt

In [19]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from string import punctuation
from preprocess_twitter import tokenize as tokenizer_g
from gensim.parsing.preprocessing import STOPWORDS
from preprocess_twitter import tokenize as tokenizer_g
#skplt.plot_confusion_matrix(y, preds)
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Input, LSTM
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Input, Merge, Convolution1D, MaxPooling1D, GlobalMaxPooling1D
import numpy as np
import pdb
from nltk import tokenize
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.model_selection import KFold
from keras.utils import np_utils
from string import punctuation
import codecs
import operator
import gensim, sklearn
from collections import defaultdict
from batch_gen import batch_gen
import sys
from string import punctuation
from preprocess_twitter import tokenize as tokenizer_g
from gensim.parsing.preprocessing import STOPWORDS
from nltk import tokenize as tokenize_nltk

Using TensorFlow backend.


In [2]:
#json reading using pandas
tweets_pd = pd.read_json('classified_tweets_latest.json', orient='columns')
tweets_pd.head()
tweets_text = (tweets_pd['tweet'].apply(lambda tweet : eval(tweet))).apply(lambda tweet : tweet['text'])
tweets_pd = tweets_pd.assign(tweets_text = tweets_text)

In [3]:
tweets_pd.head()

,borne,class,conf,count,exercise,food,label_data,runid,stamp,status,statusvec,time,tweet,tweetid,userid,tweets_text
0,None,FOOD,1,1,None,1,Junk,1,2017-01-26 06:37:29,@NicholasUnder @machineiv dip them alive in m...,None,2017-06-05 22:49:46.662565,"{'retweet_count': 0, 'favorited': False, 'crea...",824506529873350656,jason_a,@NicholasUnder @machineiv dip them alive in m...
1,None,FOOD,1,1,None,1,Unhealthy,1,2017-01-26 06:37:30,Looking forward to the weekend so i can eat my...,None,2017-06-05 22:49:47.360769,"{'retweet_count': 0, 'favorited': False, 'crea...",824506531941117952,jason_a,Looking forward to the weekend so i can eat my...
2,None,FOOD,1,1,None,1,Junk,1,2017-01-26 06:37:30,Baking soda that is,None,2017-06-05 22:50:17.088828,"{'retweet_count': 0, 'favorited': False, 'crea...",824506533295775744,jason_a,Baking soda that is
3,None,FOOD,1,1,None,1,Unhealthy,1,2017-01-26 06:37:30,I need some freakin Oreos,None,2017-06-05 22:50:17.759452,"{'retweet_count': 0, 'favorited': False, 'crea...",824506534029778944,jason_a,I need some freakin Oreos
4,None,FOOD,1,1,None,1,Junk,1,2017-01-26 06:37:31,Set of 10 Furniture Pumpkin Knob https://t.co...,None,2017-06-05 22:50:18.382275,"{'retweet_count': 0, 'extended_entities': {'me...",824506535233593344,jason_a,Set of 10 Furniture Pumpkin Knob https://t.co...


In [4]:
#tweets_pd.label_data.value_counts()
#missing values checking for label_data
tweets_pd = tweets_pd.loc[tweets_pd['food'] != 'None']
tweets_pd = tweets_pd.loc[tweets_pd['label_data'] != '']
print(tweets_pd.shape)

(7294, 16)


In [5]:
tweets_pd['label_data'] = tweets_pd.label_data.str.lower()
tweets_pd['tweets_text'] = tweets_pd.tweets_text.apply(lambda x: ' '.join(re.sub(r"([!@#$]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)",'',x).split()))

In [6]:
#after removing special characters, we left with plain data with alphanumeric characters
#seperate the data based on their class
tweets_pd_food = tweets_pd.loc[tweets_pd.food == '1']
tweets_pd_borne = tweets_pd.loc[tweets_pd.borne == '1']
tweets_pd_exercies = tweets_pd.loc[tweets_pd.exercise == '1']

In [7]:
print("tweets with food dataframe shape is {}".format(tweets_pd_food.shape))
print("tweets with borne dataframe shape is {}".format(tweets_pd_borne.shape))
print("tweets with exercises dataframe shape is {}".format(tweets_pd_exercies.shape))

tweets with food dataframe shape is (7294, 16)
tweets with borne dataframe shape is (470, 16)
tweets with exercises dataframe shape is (2201, 16)


In [8]:
pd_792017 = pd.read_json('07292017.json', orient='columns')
pd_8022017 = pd.read_json('08022017.json', orient='columns')
pd_8072017 = pd.read_json('08072017.json', orient='columns')
pd_8082017 = pd.read_json('08082017.json', orient='columns')
pd_8242017 = pd.read_json('08242017.json', orient='columns')

In [9]:
tweets_data = pd.concat([pd_792017, pd_8022017, pd_8072017, pd_8082017, pd_8242017], ignore_index=True)
tweets_data_text = (tweets_data.tweet.apply(lambda tweet: eval(tweet))).apply(lambda text: text['text'])

In [10]:
#data with 9 columns
tweets_data = tweets_data.assign(tweets_data_text = tweets_data_text)
#tweets_data_text.head()
#change the labels into category codes
tweets_data.food = tweets_data.food.astype("category")

In [11]:
print(tweets_data.shape)
#remove the tweets without label_data
print("data with labels {}".format(sum(tweets_data['label_data'] != '')))
print("removing data without labels")
#missing values checking for label_data
tweets_data = tweets_data.loc[tweets_data['label_data'] != '']
print("after removing the shape of the data is{}".format(tweets_data.shape))

(28246, 10)
data with labels 26922
removing data without labels
after removing the shape of the data is(26922, 10)


In [12]:
tweets_data['label_data'] = tweets_data.label_data.str.lower()
tweets_data['tweets_text'] = tweets_data.tweets_data_text.apply(lambda x: ' '.join(re.sub(r"([!@#$]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)",'',x).split()))
print(tweets_data.tweets_data_text.head())

0    RT @_skull_queen_: So..... has anyone ever act...
1    RT @_skull_queen_: So..... has anyone ever act...
2    RT @_skull_queen_: So..... has anyone ever act...
3    RT @_skull_queen_: So..... has anyone ever act...
4    RT @_skull_queen_: So..... has anyone ever act...
Name: tweets_data_text, dtype: object


In [13]:
#after removing special characters, we left with plain data with alphanumeric characters
#seperate the data based on their class
tweets_data_food = tweets_data.loc[tweets_data.food == 1]
tweets_data_borne = tweets_data.loc[tweets_data.borne == 1]
tweets_data_exercies = tweets_data.loc[tweets_data.exercise == 1]

In [14]:
tweets_data_food = tweets_data_food[['label_data','tweets_text']]
tweets_data_borne = tweets_data_borne[['label_data','tweets_text']]
tweets_data_exercies = tweets_data_exercies[['label_data','tweets_text']]
tweets_pd_food = tweets_pd_food[['label_data','tweets_text']]
tweets_pd_borne = tweets_pd_borne[['label_data','tweets_text']]
tweets_pd_exercies = tweets_pd_exercies[['label_data','tweets_text']]

In [15]:
tweets_food = pd.concat([tweets_data_food, tweets_pd_food])
tweets_borne = pd.concat([tweets_data_borne, tweets_pd_borne])
tweets_exercies = pd.concat([tweets_data_exercies, tweets_pd_exercies])

In [16]:
print("tweets with food dataframe shape is {}".format(tweets_food.shape))
print("tweets with borne dataframe shape is {}".format(tweets_borne.shape))
print("tweets with exercises dataframe shape is {}".format(tweets_exercies.shape))

tweets with food dataframe shape is (19504, 2)
tweets with borne dataframe shape is (16550, 2)
tweets with exercises dataframe shape is (2469, 2)


In [17]:
#'healthy','unhealthy','junk','relevant','irrelevant'
food_labels = ['healthy','unhealthy','junk']
borne_labels = exercise_labels = ['junk','relevant','irrelevant']

In [18]:
tweets_food = tweets_food.loc[tweets_food.label_data.isin(food_labels)]
tweets_borne = tweets_borne.loc[tweets_borne.label_data.isin(borne_labels)]
tweets_exercies = tweets_exercies.loc[tweets_exercies.label_data.isin(exercise_labels)]

#tweets_data_food.label_data.value_counts()

In [20]:
def glove_tokenize(text):
    text = tokenizer_g(text)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    return words

In [21]:
tweets_food.columns

Index(['label_data', 'tweets_text'], dtype='object')

In [22]:
print(tweets_food.tweets_text[0])
#glove_tokenize(tweets_pd.tweets_text[0])

NicholasUnder machineiv dip them alive in melted chocolate and encase em Cover them in seeds and peanut butter and let birds eat em 25


In [23]:
#word2vec_model = gensim.models.Word2Vec.load_word2vec_format('glove.twitter.27B.25d.txt')
embeddings_index = {}
f = open('glove.twitter.27B.25d.txt')
for line in f:
    #print(line)
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [27]:
MAX_NB_WORDS = 20000
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(tweets_food.tweets_text)
sequences = tokenizer.texts_to_sequences(tweets_food.tweets_text)

/home/santhoshm/.local/lib/python3.5/site-packages/keras/preprocessing/text.py:145: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [28]:
MAX_SEQUENCE_LENGTH = max(map(lambda x:len(x), tweets_food.tweets_text))
MAX_SEQUENCE_LENGTH

143

In [29]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 18230 unique tokens.


In [51]:
data.shape

(17894, 143)

In [55]:
EMBEDDING_DIM = 25

In [56]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [32]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 18230 unique tokens.


In [33]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [34]:
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout
from keras import backend as K
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, Lambda

In [35]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(256, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(512, 5, padding="same", activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35, padding="same")(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
pred = Dense(3, activation='softmax', name = 'fc')(l_dense)

In [36]:
#ot = K.concatenate([l_dense, preds], axis=1)

In [37]:
model = Model(sequence_input, pred)

In [38]:
# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['acc'])

In [39]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
# VALIDATION_SPLIT = 0.2
# labels = np.asarray(tweets_food.label_data)
# num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

# x_train = data[:-num_validation_samples]
# y_train = labels[:-num_validation_samples]
# x_val = data[-num_validation_samples:]
# y_val = labels[-num_validation_samples:]
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [41]:
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(tweets_food['label_data'])
#print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
labels = onehot_encoder.fit_transform(integer_encoded)
#print(tweets_food['label_data'])

In [42]:
print(labels)

[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 ..., 
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]]


In [43]:
tweets_food['label_data'].head()

236    unhealthy
237    unhealthy
242      healthy
243      healthy
259    unhealthy
Name: label_data, dtype: object

In [44]:
tweets_food.label_data.value_counts()

healthy      8316
unhealthy    6448
junk         3130
Name: label_data, dtype: int64

In [45]:
type(labels)

numpy.ndarray

In [46]:
from sklearn.cross_validation import train_test_split

/home/santhoshm/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [47]:
train_x, test_x, train_y, test_y = train_test_split(data, labels, test_size = 0.2)

In [48]:
x_train, val_x, y_train, val_y = train_test_split(train_x, train_y, test_size = 0.2) 

In [49]:
model.fit(x_train, y_train, epochs = 2, batch_size = 32)

Epoch 1/2
11452/11452 [==============================] - 14s 1ms/step - loss: 1.0049 - acc: 0.4986
Epoch 2/2
11452/11452 [==============================] - 12s 1ms/step - loss: 0.9624 - acc: 0.5260


In [48]:
pred_y = model.predict(test_x)

In [49]:
np.argmax(pred_y, axis=1)

array([0, 2, 0, ..., 0, 0, 0])

In [50]:
from sklearn.metrics import f1_score

In [52]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
tweets_food['label_data'] = encoder.fit_transform(tweets_food['label_data'])
encoder.classes_

array(['healthy', 'junk', 'unhealthy'], dtype=object)

In [54]:
onehot_encoder.test_y

array([[ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       ..., 
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.]])

#word2vec genism

In [116]:
from gensim.models.word2vec import Word2Vec

In [129]:
#tokenize text into array of words
def tokenizeData(pdSeries):
    #corpus = pdSeries.apply(lambda x: x.replace("\n","").split())
    corpus = [z.lower().replace('\n','').split() for z in pdSeries]
    return corpus

In [155]:
tweet_text = tokenizeData(tweets_exercies.tweets_text)

In [206]:
#train word2vec on each tweet text
n_dim = 250
#Initialize model and build vocab
tweet2vec = Word2Vec(size=n_dim, min_count=5)
tweet2vec.build_vocab(tweet_text)

In [207]:
tweet2vec.train(tweet_text)

114931

In [208]:
#tweet2vec.vocab

In [209]:
#Build word vector for training set by using the average value of all word vectors in the tweet, then scale
def buildWord2Vec(text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += tweet2vec[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [210]:
from sklearn.preprocessing import scale
train_vecs = np.concatenate([buildWord2Vec(z, n_dim) for z in tweet_text])
train_vecs = scale(train_vecs)

#Train word2vec on test tweets
#imdb_w2v.train(x_test)

In [211]:
lb = LabelEncoder()
y = lb.fit_transform(tweets_exercies.label_data)

In [212]:
train_x, test_x, train_y, test_y = train_test_split(train_vecs, y, test_size = 0.2)

In [213]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score

maxiter = 1000
batch = 150
nnet = MLPClassifier(alpha = 1e-8, solver = 'adam', batch_size=batch)
nnet.fit(train_x, train_y)

print("Test Accuracy: {:.2f}".format(accuracy_score(nnet.predict(test_x), test_y)))

Test Accuracy: 0.49


In [214]:
#random forest
# Initialize a Random Forest classifier with 500 trees
exercises_forest = RandomForestClassifier(n_estimators = 500, criterion='entropy', n_jobs = -1, random_state = 1) 
exercises_forest.fit(train_x, train_y)
pred = exercises_forest.predict(test_x)
print('f1 score of RF is {}'.format(f1_score(test_y, pred, average='weighted')))
print('precision score of RF is {}'.format(precision_score(test_y, pred, average="weighted")))

f1 score of RF is 0.6013631688384189
precision score of RF is 0.6184096771718968


In [216]:
from sklearn.svm import SVC
exercises_svm = SVC(kernel='rbf', C=10, gamma = 1)
exercises_svm.fit(train_x, train_y)
pred = exercises_svm.predict(test_x)
print("accuracy of the model is {}".format(exercises_svm.score(test_x, test_y)))
print('f1 score of SVM is {}'.format(f1_score(test_y, pred, average='weighted')))
print('precision score of SVM is {}'.format(precision_score(test_y, pred, average="weighted")))

accuracy of the model is 0.5938775510204082
f1 score of SVM is 0.5868880660675946
precision score of SVM is 0.5882995745591738


In [218]:
from random import shuffle

In [217]:
#doc2vec
LabeledSentence = gensim.models.doc2vec.LabeledSentence

Word2Vec simply converts a word into a vector.

Doc2Vec not only does that, but also aggregates all the words in a sentence into a vector. To do that, it simply treats a sentence label as a special word, and does some voodoo on that special word. Hence, that special word is a label for a sentence.

In [227]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in enumerate(tweets):
        label = "{}_{}".format(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

In [236]:
lb = LabelEncoder()
food_labels = lb.fit_transform(tweets_food.label_data)

In [309]:
train_x, test_x, train_y, test_y = train_test_split(tweets_food.tweets_text, food_labels, test_size = 0.2)

In [400]:
#tokenize text into array of words
def tokenizeData( tokens_only, pdSeries,lab):
    #corpus = pdSeries.apply(lambda x: x.replace("\n","").split())
    #corpus = [np.array(z.lower().replace('\n','').split()) for z in pdSeries]
    for i, line in enumerate(pdSeries):
        if tokens_only:
            yield gensim.utils.simple_preprocess(line)
        else:
             # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), ["".join([lab,str(i)])])
    #return corpus

In [311]:
# import gensim
# def read_corpus(fname, tokens_only=False):
#     with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
#         for i, line in enumerate(f):
#             if tokens_only:
#                 yield gensim.utils.simple_preprocess(line)
#             else:
#                 # For training data, add tags
#                 yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [401]:
tweet_food_text_train = list(tokenizeData(False, train_x, "train"))
tweet_food_text_test = list(tokenizeData(False, test_x, "test"))

In [402]:
#tweet_food_text

In [403]:
print(train_x.shape)
print(train_y.shape)

(14315,)
(14315,)


In [404]:
# x_train = labelizeTweets(train_x, 'TRAIN')
# x_test = labelizeTweets(test_x, 'TEST')

Next we instantiate our two Doc2Vec models, DM and DBOW. The gensim documentation suggests training over the data multiple times and either adjusting the learning rate or randomizing the order of input at each pass. We then collect the movie review vectors learned by the models

In [405]:
#instantiate our DM and DBOW models
size = 300
model_dm = gensim.models.Doc2Vec(min_count=1, window=10, size=size, sample=1e-3, negative=5, workers=3)
#DBOW
model_dbow = gensim.models.Doc2Vec(min_count=1, window=10, size=size, sample=1e-3, negative=5, dm=0, workers=3)

In [406]:
tweets_food_text_model = [item for pair in zip(tweet_food_text_train, tweet_food_text_test) for item in pair]
#to add list just use + operatr

In [407]:
#build vocab over all tweets
model_dm.build_vocab(tweet_food_text_train+tweet_food_text_test)
model_dbow.build_vocab(tweet_food_text_train+tweet_food_text_test)

In [408]:
#

In [409]:
#model_dm.train(tweet_food_text_train, total_examples=model_dm.corpus_count, epochs=model_dm.iter)
#model_dbow.train(tweet_food_text_train, total_examples=model_dbow.corpus_count, epochs=model_dbow.iter)
perm = np.random.permutation(train_x.shape[0], )
train_x.shape

(14315,)

In [410]:
#We pass through the data set multiple times, shuffling the training reviews each time to improve accuracy.
all_train_tweets = tweet_food_text_train
for epoch in range(10):
      
    model_dm.alpha -= 0.002
    model_dbow.alpha -= 0.002
   #model.min_alpha = model.alpha
    model_dm.train(all_train_tweets)
    model_dbow.train(all_train_tweets)

In [411]:
for epoch in range(10):
    model_dm.alpha -= 0.002
    model_dbow.alpha -= 0.002
   #model.min_alpha = model.alpha
    model_dm.train(tweet_food_text_test)
    model_dbow.train(tweet_food_text_test)

In [412]:
#Get training set vectors from our models
def getVecs(model, corpus, size):
    vecs = [np.array(model[z[0]]) for z in corpus]
    return np.concatenate(vecs)

In [418]:
#train_vecs_dm = getVecs(model_dm, tweet_food_text_train, size)
#np.array(model_dm[np.asarray(tweet_food_text_train[0])[0]])
#tweet_food_text_train[0][0]
#train_vecs_dm.shape
#model_dm.most_similar("money")
len(model_dm.docvecs["test0"])
train_vecs_dm = [model_dm.docvecs["".join(["train",str(i)])] for i, line in enumerate(train_x)]
test_vecs_dm = [model_dm.docvecs["".join(["test",str(i)])] for i, line in enumerate(test_x)]
train_vecs_dbow = [model_dbow.docvecs["".join(["train",str(i)])] for i, line in enumerate(train_x)]
test_vecs_dbow = [model_dbow.docvecs["".join(["test",str(i)])] for i, line in enumerate(test_x)]

In [419]:
len(test_vecs_dm)

3579

In [420]:
train_vecs = np.hstack((train_vecs_dm, train_vecs_dbow))
test_vecs = np.hstack((test_vecs_dm, test_vecs_dbow))

In [421]:
#random forest
# Initialize a Random Forest classifier with 500 trees
food_forest = RandomForestClassifier(n_estimators = 500, criterion='entropy', n_jobs = -1, random_state = 1) 
food_forest.fit(train_vecs, train_y)
pred = food_forest.predict(test_vecs)
print('f1 score of RF is {}'.format(f1_score(test_y, pred, average='weighted')))
print('precision score of RF is {}'.format(precision_score(test_y, pred, average="weighted")))

f1 score of RF is 0.33922807981954034
precision score of RF is 0.3094953743463355


/home/santhoshm/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/santhoshm/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [422]:
maxiter = 1000
batch = 150
nnet = MLPClassifier(alpha = 1e-8, solver = 'adam', batch_size=batch)
nnet.fit(train_vecs, train_y)

print("Test Accuracy: {:.2f}".format(accuracy_score(nnet.predict(test_vecs), test_y)))

Test Accuracy: 0.35


In [427]:
model_dbow.similar_by_word("left")

[('fishs', 0.23842597007751465),
 ('kinda', 0.2048811912536621),
 ('urge', 0.20292916893959045),
 ('cody', 0.20092925429344177),
 ('vital', 0.20052559673786163),
 ('lfo', 0.1966307908296585),
 ('memeiorde', 0.19456884264945984),
 ('drugs', 0.19377486407756805),
 ('danyell', 0.19292503595352173),
 ('ready', 0.1897403597831726)]